In [157]:
import numpy as np
import math

# plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [158]:
def neural_model(x_in, dt, p):

    x_out = np.zeros(x_in.shape)

    # define function constants
    FR_MAX = 2.5
    V0 = 6
    SIGMOID_SLOPE = 0.56

    # coupling coeffecients
    W_pe = 900
    W_pi = 740
    W_ep = 1000
    W_ip = 280
    
    # time constants
    TAU_E = 0.01
    TAU_I = 0.02

    # get current states
    v_pe = x_in[0,]
    z_pe = x_in[1,]
    v_pi = x_in[2,]
    z_pi = x_in[3,]
    v_ep = x_in[4,]
    z_ep = x_in[5,]
    v_ip = x_in[6,]
    z_ip = x_in[7,]
    
    v_e = v_ep + p
    v_i = v_ip
    v_p = v_pe - v_pi
    
    # Sigmoid functions
    f_p = FR_MAX / (1 + math.exp( SIGMOID_SLOPE * (V0 - v_p) ))
    f_e = FR_MAX / (1 + math.exp( SIGMOID_SLOPE * (V0 - v_e) ))
    f_i = FR_MAX / (1 + math.exp( SIGMOID_SLOPE * (V0 - v_i) ))

    # state update equation - propagate states
    a = 1 / TAU_E
    b = 1 / TAU_I
    
    x_out[0] = v_pe + dt * z_pe
    x_out[1] = z_pe + dt * (W_pe*a*f_e - 2*a*z_pe - a**2 * v_pe)
    x_out[2] = v_pi + dt * z_pi
    x_out[3] = z_pi + dt * (W_pi*b*f_i - 2*b*z_pi - b**2 * v_pi)
    x_out[4] = v_ep + dt * z_ep
    x_out[5] = z_ep + dt * (W_ep*a*f_p - 2*a*z_ep - a**2 * v_ep)
    x_out[6] = v_ip + dt * z_ip
    x_out[7] = z_ip + dt * (W_ip*a*f_p - 2*a*z_ip - a**2 * v_ip)
    
    return x_out

In [162]:
# define simulation time, t
T = 10
Fs = 1000
N = T * Fs
dt = 1/Fs

t = np.arange(0,N*dt,dt)
        
# Initialise neural mass model
N_STATES = 8
x = np.zeros((N_STATES,N))

# Define Gaussian noise input, p(t)
mu = 90
sigma = 50
cov = np.zeros((N_STATES, N_STATES))
cov[1, 1] = sigma**2

p = np.random.multivariate_normal(np.zeros(N_STATES).tolist(), cov, N).T


In [163]:
fig = go.Figure()
fig.add_trace(
        go.Scatter(x=t, y=p[1,:], mode='lines', name='likelihood',
                   line={'color': 'rgb(115,115,115)'}))


In [165]:
# Numerical forward simulation using Euler-Maruyama integration

for n in range(0,N-1):
    x[:,n+1] = neural_model(x[:,n], dt, mu) + p[:,n]
    
# Plot simulated EEG
H = np.array([1, 0, -1, 0, 0, 0, 0, 0])
y = np.matmul(H, x)
    
fig = go.Figure()
fig.add_trace(
        go.Scatter(x=t, y=y, mode='lines', name='likelihood',line={'color': 'rgb(0,0,0)'}))

fig.update_layout(xaxis_title='Time (s)', yaxis_title='EEG (mV)')

In [188]:
# check frequency power spectrum
sp = np.fft.fft(y)
freq = np.fft.fftfreq(y.shape[-1])

F = np.fft.rfft(y)
P2 = np.abs(F/N)
P1 = P2[0:int(N/2)]
P1[1:] = 2*P1[1:]

f = Fs*np.arange(0,N/2,1)/N

fig = go.Figure()
fig.add_trace(
        go.Scatter(x=f, y=P1, mode='lines', name='likelihood',line={'color': 'rgb(0,0,0)'}))

fig.update_layout(xaxis_title='f (Hz)', yaxis_title='|P1(f)|')
